In [12]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 28 07:08:16 2024


@author: Faiza
"""

from abc import ABC, abstractmethod
import numpy as np
import struct
from typing import Tuple
from bitarray import bitarray
# from src.compressors.compressor import Compressor

import pandas as pd

import torch
from tqdm import tqdm
from load_dataset import Dataset
import os

import time
# import matplotlib.pyplot as plt
# from PIL import Image
import cv2
import sys
from datetime import datetime


import math
# from src.compressors.compressor import Compressor
import struct


from bitarray import bitarray
from bitarray.util import int2ba, ba2int

In [13]:


class Compressor(ABC):
    @abstractmethod
    def encode(self, array: np.ndarray) -> bytes:
        pass
    @abstractmethod
    def decode(self, array: bytes) -> np.ndarray:
        pass


# Existing Quantization schemes

In [14]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/qsgd.jl")


class QSGD(Compressor):
    def __init__(self, s: int, zero_rle: bool, type=None):
        self.type = type
        if self.type == "LFL":
            self.s = 2
        else:
            self.s = s
        self.zero_rle = zero_rle

    # format:    | length | norm | s(1) | sign(1) | s(2) | ... | sign(n) | 
    # (no 0-rle) | 32     | 32   | ?    | 1       | ?    | ... | 1       |
    # format:    | length | norm | n_zeros | s(1) | sign(1) | n_zeros | s(2) | ... | sign(n) |
    # (0-rle)    | 32     | 32   | ?       | ?    | 1       | ?       | ?    | ... | 1       |
    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_qsgd(array, self.s, self.type, seed, cid, client_name)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_qsgd(array, self.s, self.type, use_lo_quant)
        return result
    
#%% testing qsgd quantization

print("QSGD")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = QSGD(2,True)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")



QSGD
Before Quantization:  [3.0488355 5.28843   1.1547178 2.564842  9.7672825 1.3771464 1.1520959
 7.613091  2.8873534 0.1299076 6.625842  6.5229917 2.7006006 0.289284
 3.561608  7.1835585]
Encoded b'&\x00\x00\x00\x9d\x83\x99A"\x93\xb2J\x01\x00\x00\x00'
After Quantization:  [9.594632 9.594632 0.       0.       9.594632 0.       9.594632 0.
 0.       0.       0.       0.       9.594632 0.       9.594632 0.      ]
52.88461538461539% smaller 
70.83333333333333% smaller 


In [15]:
#%%SETTING UP Julia and gzip Class
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/gzip.jl")

import numpy as np

class GZip(Compressor):
    def __init__(self, s: int):
        self.s = s

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_gzip(array, self.s, seed)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_gzip(array, self.s)
        return result


#%% testing gzip quantization
print("GZip")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = GZip(1)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")


GZip
Before Quantization:  [8.089339   7.1322646  7.5165095  1.5927403  2.3870368  8.061734
 1.2486603  0.23253496 9.789648   6.1566253  0.5452636  5.2661214
 4.298582   2.14563    5.094013   7.1632986 ]
Encoded b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\ncd\x80\x00FF\x08\x9d8\x7f\x8b#\x00uF>\xc4\x14\x00\x00\x00'
After Quantization:  [22.577822  0.        0.        0.        0.        0.        0.
  0.       22.577822 22.577822  0.        0.        0.        0.
  0.        0.      ]
38.46153846153846% smaller 
61.904761904761905% smaller 


In [16]:
#%%FP8

# print("Starting up Julia.")
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/fp8.jl")
# print("Finished starting up Julia.")

FP8_FORMAT = (1, 5, 2)
FP32_FORMAT = (1, 8, 23)

def get_emax(format):
    return (2**(format[1]-1)) - 1

def get_emin(format):
    return 1 - get_emax(format)


class FP8(Compressor):
    def __init__(self):
        self.fp8s_repr_in_fp32 = []
        self.fp8s = []
        self.s = -1
        # negative values before positive values.
        def insert(num):
            byte = struct.pack('>B', num)
            [num] = self.decode(byte)
            if not np.isnan(num):
                self.fp8s.append(byte)
                self.fp8s_repr_in_fp32.append(num)
                bits = bitarray()
                bits.frombytes(byte)
        for i in list(reversed(range(128, 253))) + list(range(0, 128)):
            insert(i)
        self.fp8s_repr_in_fp32 = np.array(self.fp8s_repr_in_fp32).astype(np.float32)

    def get_fp8_neighbors(self, f: np.float32) -> Tuple[bytes, bytes]:
        idx_high = np.searchsorted(self.fp8s_repr_in_fp32, f, side='right')
        idx_low = idx_high - 1
        if idx_high == len(self.fp8s_repr_in_fp32):
            idx_high -= 1
        return self.fp8s[idx_low], self.fp8s[idx_high], self.fp8s_repr_in_fp32[idx_low], self.fp8s_repr_in_fp32[idx_high]

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_fp8(array, self.fp8s_repr_in_fp32, self.fp8s, seed)
        return bytes(result)
    
    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_fp8(array)
        return result
#%% testing fp32 quantization
print("FP8")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = FP8()
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")


FP8
Before Quantization:  [0.3520813  8.492879   4.1909027  3.386259   9.799349   0.13562189
 9.442938   8.533109   4.028142   5.449075   4.448057   7.3852687
 4.5708385  5.6423573  2.50317    3.7957022 ]
Encoded b'6HDCI0IHDFDGEEAC'
After Quantization:  [ 0.375  8.     4.     3.5   10.     0.125 10.     8.     4.     6.
  4.     7.     5.     5.     2.5    3.5  ]
52.88461538461539% smaller 
70.83333333333333% smaller 


In [17]:

import cv2


class customCompression():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    



#%%

# data = np.random.rand(1,16)

data = list(np.random.rand(1,16).flatten()*10)
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint8
cmp = customCompression(data_type)
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))


[7.2863574  1.7006388  7.4450507  1.1426399  7.7994404  3.738008
 1.531081   8.656741   6.7041945  4.1158304  0.01969053 8.971472
 4.8029075  8.04605    3.391842   6.2828546 ]
Encoded [206  47 211  31 221 105  43 246 190 116   0 255 136 228  96 178]
After Quantization:  [7.25132558 1.66962668 7.42685071 1.10794629 7.77790095 3.70571811
 1.52920658 8.65552657 6.68964519 4.09187338 0.01969053 8.97147179
 4.79397387 8.02363612 3.38977288 6.2683849 ]
<class 'numpy.uint8'> 48.275862068965516% smaller 
<class 'numpy.uint8'> 28.571428571428573% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  0.00042605671999475453


In [18]:
print(sys.getsizeof(data))
print(sys.getsizeof(dec))
print(sys.getsizeof(enc))

168
232
120


# Custom Frequency Domain Quantization

In [20]:
from scipy.fftpack import dct, idct

class customFreq_1_point_5():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized
    

data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_1_point_5()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.79331183 0.10158886 0.5699902  0.51827    0.9152226  0.14632188
 0.10714582 0.730827   0.91277975 0.9289109  0.44376284 0.93530905
 0.47954693 0.1905915  0.7572435  0.9289565 ]
Encoded [ 0.9453   0.465    0.638    0.769    0.991    0.5947   0.1343   0.9727
  0.9824   0.888    0.2603   0.623    0.1465  -0.03091  0.584    0.355  ]
After Quantization:  [0.7932832  0.10166854 0.56992364 0.51807475 0.91541064 0.14623436
 0.10726646 0.7309777  0.912605   0.9289638  0.4437859  0.9353111
 0.47956133 0.19059592 0.75732327 0.9289014 ]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  1.0845411e-08


In [21]:
from scipy.fftpack import dct, idct

class customFreq_singleDCT():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(x, axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(x, axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized
    

data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_singleDCT()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.40562192 0.22467376 0.5786193  0.13976963 0.40958834 0.21521123
 0.05198686 0.42317915 0.4895062  0.7614812  0.55971885 0.42597938
 0.6300761  0.643463   0.07666218 0.45017865]
Encoded [ 1.621   -0.256   -0.087    0.4246  -0.0661  -0.1614  -0.12354  0.1631
  0.0655  -0.2412   0.178   -0.2169   0.3103   0.207    0.251   -0.03134]
After Quantization:  [0.40552542 0.22451337 0.57851917 0.1396526  0.40950578 0.21514195
 0.05195443 0.42310628 0.48937997 0.7613667  0.5596446  0.42588633
 0.62997025 0.64350104 0.07656485 0.450142  ]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  8.964298e-09


# Custom Freq 3 (CustomFreq + Custom2 but more sophisticated)

In [22]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)
        return enc
    
    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_3():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [np.uint32, np.uint16, np.uint8]):
        self.precision_levels = precision_levels
        self.linearQuant_0 = Linear_quant(self.precision_levels[0])
        self.linearQuant_1 = Linear_quant(self.precision_levels[1])
        self.linearQuant_2 = Linear_quant(self.precision_levels[2])
        
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        self.mean_importance = np.mean(importance)
        # if self.mean_importance > 0.1:
        enc = self.linearQuant_0.encode(weights_f)  # 8-bit
        precision = int(str(self.linearQuant_0.quantization_type).split('t')[-1].split('\'')[0])
        print(precision)

        # elif self.mean_importance > 0.01:
        #     enc = self.linearQuant_1.encode(weights_f)   # 4-bit
        #     precision = int(str(self.linearQuant_1.quantization_type).split('t')[-1].split('\'')[0])
        #     # enc = self.quantize(weights_f, 16)
        # else:
        #     enc = self.linearQuant_2.encode(weights_f)   # 2-bit
        #     precision = int(str(self.linearQuant_2.quantization_type).split('t')[-1].split('\'')[0])

        
        # Quantize frequency components
        # enc = self.quantize(enc, precision)
        return enc


    def decode(self, enc) :

        # if self.mean_importance > 0.1:
        weights_quantized = self.linearQuant_0.decode(enc)  # 8-bit
        # elif self.mean_importance > 0.01:
        #     weights_quantized = self.linearQuant_1.decode(enc)   # 4-bit
        # else:
        #     weights_quantized = self.linearQuant_2.decode(enc)   # 2-bit
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_3()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.20517802 0.3858592  0.5043621  0.16739534 0.8572458  0.40283382
 0.7192395  0.3937499  0.88723356 0.59849674 0.22572844 0.8410991
 0.8650191  0.64502114 0.60185325 0.5507706 ]
32
Encoded [1633348028 1593892085 3952755489 1470550537 4294967295 3189105010
 3362035082 2704825469 3411096099 2930258382  456073978 3422719434
 2367476512  623489999  283488008          0]
After Quantization:  [0.20517798 0.38585929 0.50436212 0.1673954  0.85724583 0.40283394
 0.71923949 0.39374987 0.88723341 0.5984968  0.22572847 0.84109906
 0.86501908 0.64502111 0.60185327 0.55077065]
<class 'numpy.uint16'> 27.586206896551722% smaller 
<class 'numpy.uint16'> 0.0% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  4.0783338128445065e-15


# CustomFreq2.5

In [24]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_2_5():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        self.linearQuant = Linear_quant()
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(x, axis=-1, norm='ortho')
    def inverse_dct_transform(self, x):
        return idct(x, axis=-1, norm='ortho')
    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_1_point_5()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.96453446 0.21308406 0.46667728 0.84674025 0.86505586 0.71669483
 0.91899395 0.43126684 0.7287961  0.1371453  0.08365608 0.48049384
 0.3407544  0.9439819  0.6283354  0.630267  ]
Encoded [ 1.03     0.6724   0.534    0.955    1.124    0.77     0.92     0.4006
  0.4749   0.10706 -0.03928  0.571    0.317    0.7266   0.1387   0.07666]
After Quantization:  [0.96452683 0.21315369 0.46680143 0.8468318  0.8652445  0.71686375
 0.9191998  0.4313223  0.7289357  0.13712552 0.08366581 0.48075968
 0.34089786 0.9439382  0.6283739  0.6302836 ]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  1.5823545e-08


# Custom Freq 2 (CustomFreq + Custom2) 
- best version

In [25]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_2():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        self.linearQuant = Linear_quant()
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_2()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.86064    0.79101723 0.28225443 0.04652305 0.0314031  0.8413465
 0.5733758  0.80970156 0.82406324 0.30854437 0.5234223  0.9601614
 0.4289598  0.34485024 0.02361145 0.34724194]
Encoded [2887048623 4294967295 2507879817 1727873875 1107998923 3612148086
 3392218656 3023174670 3204009258 1299379639 1754318578 2709335434
  476273705  291133443          0 1212410894]
After Quantization:  [0.86064001 0.79101712 0.28225442 0.04652304 0.03140304 0.84134638
 0.57337575 0.80970155 0.82406309 0.30854429 0.52342217 0.96016134
 0.42895969 0.34485024 0.02361142 0.34724191]
<class 'numpy.uint16'> 27.586206896551722% smaller 
<class 'numpy.uint16'> 0.0% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  5.921788274685641e-15


# Federated Implementation

In [37]:
#%% PAQ with Compression


class MNIST_PAQ_COMP:
	def __init__(self, cmp_list, filename="saved_models", number_of_clients=1, aggregate_epochs=10, local_epochs=5, precision=7, r=1.0):
		self.model = None
		self.criterion = torch.nn.CrossEntropyLoss()
		self.optimizer = None
		self.number_of_clients = number_of_clients
		self.aggregate_epochs = aggregate_epochs
		self.local_epochs = local_epochs
		self.precision = precision
		self.r = r
		self.filename = filename
		self.compression_ratio = 0
		self.cmp = None
		self.cmp_list = cmp_list

	def define_model(self):
		self.model = torch.nn.Sequential(
			torch.nn.Conv2d(1, 2, kernel_size=5),
			torch.nn.ReLU(),
			torch.nn.Conv2d(2, 4, kernel_size=7),
			torch.nn.ReLU(),
			torch.nn.Flatten(),
			torch.nn.Linear(1296, 512),
			torch.nn.ReLU(),
			torch.nn.Linear(512, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 32),
			torch.nn.ReLU(),
			torch.nn.Linear(32, 10),
			torch.nn.Softmax(dim=1),
		)		

	def get_weights_custom(self, dtype=np.float32):
        
		precision = self.precision 
		weights = []
		start_size = 0
		end_size = 0
        
        
		for layer in self.model:			
			try:
				layer_weights = layer.weight.detach().numpy().astype(dtype)
				start_size = start_size + sys.getsizeof(layer_weights)
				# layer_weights_enc = self.cmp.encode(layer_weights)
				layer_weights_enc = self.cmp.encode(layer_weights.flatten())
				decoded_weights = self.cmp.decode(layer_weights_enc)
				decoded_weights = decoded_weights.astype(dtype)
				decoded_weights = decoded_weights.reshape(layer_weights.shape)
				end_size = end_size + sys.getsizeof(decoded_weights)
		
				layer_bias = layer.bias.detach().numpy().astype(dtype)
				# layer_bias_enc = self.cmp.encode(layer_bias)	
				layer_bias_enc = self.cmp.encode(layer_bias.flatten())			
				decoded_bias = self.cmp.decode(layer_bias_enc)
				decoded_bias = decoded_bias.astype(dtype)
				decoded_bias = decoded_bias.reshape(layer_bias.shape)
				weights.append([decoded_weights,decoded_bias])
				
			except:
				continue
		self.compression_ratio = ((start_size-end_size)/start_size)*100
		return np.array(weights),start_size,end_size



	def set_weights(self, weights):
		index = 0
		for layer_no, layer in enumerate(self.model):
			try:
				_ = self.model[layer_no].weight
				self.model[layer_no].weight = torch.nn.Parameter(weights[index][0])
				self.model[layer_no].bias = torch.nn.Parameter(weights[index][1])
				index += 1
			except:
				continue

	def average_weights(self, all_weights):
        
		all_weights = np.array(all_weights)
		all_weights = np.mean(all_weights, axis=0)
		all_weights = [[torch.from_numpy(i[0].astype(np.float32)), torch.from_numpy(i[1].astype(np.float32))] for i in all_weights]
		return all_weights




	def get_client_bw_and_traffic(self):
		bw_list = [10,50]
		bw =  np.random.choice(bw_list, size=1)

		traffic =  1
		return bw,traffic
	
	def quanization_scheme(self,bw,traffic):
		if bw>15:
			selected_cmp = self.cmp_list[0]
		else:
			selected_cmp = self.cmp_list[1]		
		return selected_cmp


	def client_generator(self, train_x, train_y):
		
		number_of_clients = self.number_of_clients
		size = train_y.shape[0]//number_of_clients
		train_x, train_y = train_x.numpy(), train_y.numpy()
		train_x = np.array([train_x[i:i+size] for i in range(0, len(train_x)-len(train_x)%size, size)])
		train_y = np.array([train_y[i:i+size] for i in range(0, len(train_y)-len(train_y)%size, size)])
		train_x = torch.from_numpy(train_x)
		train_y = torch.from_numpy(train_y)
		return train_x, train_y

	def single_client(self, dataset, weights, E):
		self.define_model()
		if weights is not None:
			self.set_weights(weights)
		self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
		for epoch in range(E):
			running_loss = 0
			for batch_x, target in zip(dataset['x'], dataset['y']):
				output = self.model(batch_x)
				loss = self.criterion(output, target)
				self.optimizer.zero_grad()
				loss.backward()
				self.optimizer.step()
				running_loss += loss.item()
			running_loss /= len(dataset['y'])
		# weights,start_size,end_size = self.get_weights()
		weights,start_size,end_size = self.get_weights_custom()		
		return weights, running_loss, start_size,end_size
	
	
	def test_aggregated_model(self, test_x, test_y, epoch):
		acc = 0
		with torch.no_grad():
			for batch_x, batch_y in zip(test_x, test_y):
				y_pred = self.model(batch_x)
				y_pred = torch.argmax(y_pred, dim=1)
				acc += torch.sum(y_pred == batch_y)/y_pred.shape[0]
		torch.save(self.model, "./"+self.filename+"/model_epoch_"+str(epoch+1)+".pt")
		return (acc/test_x.shape[0])
			

	def train_aggregator(self, datasets, datasets_test):
		local_epochs = self.local_epochs
		aggregate_epochs = self.aggregate_epochs
		os.system('mkdir '+self.filename)
		E = local_epochs
		aggregate_weights = None
		for epoch in range(aggregate_epochs):
			all_weights = []
			client = 0
			running_loss = 0
			selections = np.arange(datasets['x'].shape[0])
			np.random.shuffle(selections)
			selections = selections[:int(self.r*datasets['x'].shape[0])]
			clients = tqdm(zip(datasets['x'][selections], datasets['y'][selections]), total=selections.shape[0])
			for dataset_x, dataset_y in clients:
				bw,traffic = self.get_client_bw_and_traffic()
				self.cmp = self.quanization_scheme(bw,traffic)
				# print(self.cmp)
				dataset = {'x':dataset_x, 'y':dataset_y}
				weights, loss, start_size,end_size = self.single_client(dataset, aggregate_weights, E)
				running_loss += loss
				all_weights.append(weights)
				client += 1
				clients.set_description(str({"Epoch":epoch+1,"Loss": round(running_loss/client, 5)}))
				clients.refresh()
			aggregate_weights = self.average_weights(all_weights)
			agg_weight = self.set_weights(aggregate_weights)
			test_acc = self.test_aggregated_model(datasets_test['x'], datasets_test['y'], epoch)
			print("Test Accuracy:", round(test_acc.item(), 5))
			print("Compression Ratio ", self.compression_ratio)
			# print()
			print(f'start_size: {start_size/1024},end_size: {end_size}')
			clients.close()
		return agg_weight

#%%

In [38]:
number_of_clients = 328
aggregate_epochs = 10
local_epochs = 3
r = 0.5
current_time = datetime.now().time()
epoch_time = time.time()
tic = time.time()
filename = f"saved_models_{epoch_time}"
train_x, train_y, test_x, test_y = Dataset().load_csv()
# cmp = customCompression(np.uint8)
cmp_list = [customFreq_2(),FP8()]

# cmp = FP8()
# cmp = QSGD(2,True)
m_8 = MNIST_PAQ_COMP(cmp_list=cmp_list, filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = m_8.client_generator(train_x, train_y)
agg_weights = m_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y})
print("Time Taken: ", time.time()-tic)

  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_21032\585678726.py:66: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30115}: 100%|██████████| 164/164 [00:32<00:00,  5.11it/s]


Test Accuracy: 0.09644
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 2, 'Loss': 2.07555}: 100%|██████████| 164/164 [00:32<00:00,  5.05it/s]


Test Accuracy: 0.40457
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 3, 'Loss': 1.85966}: 100%|██████████| 164/164 [00:33<00:00,  4.93it/s]


...

Test Accuracy: 0.92549
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 9, 'Loss': 1.52937}: 100%|██████████| 164/164 [00:32<00:00,  4.98it/s]


Test Accuracy: 0.93313
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 10, 'Loss': 1.52629}: 100%|██████████| 164/164 [00:33<00:00,  4.88it/s]


Test Accuracy: 0.93639
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  345.98965525627136


# Uint16-custom2


{'Epoch': 1, 'Loss': 2.30095}: 100%|██████████| 164/164 [00:59<00:00,  2.74it/s]
Test Accuracy: 0.09963
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 2, 'Loss': 2.09561}: 100%|██████████| 164/164 [00:57<00:00,  2.84it/s]
Test Accuracy: 0.36056
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 3, 'Loss': 1.86639}: 100%|██████████| 164/164 [00:58<00:00,  2.81it/s]
Test Accuracy: 0.63837
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 4, 'Loss': 1.77721}: 100%|██████████| 164/164 [00:56<00:00,  2.90it/s]
Test Accuracy: 0.68152
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 5, 'Loss': 1.74531}: 100%|██████████| 164/164 [01:01<00:00,  2.68it/s]
Test Accuracy: 0.71413
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 6, 'Loss': 1.72985}: 100%|██████████| 164/164 [00:44<00:00,  3.71it/s]
Test Accuracy: 0.73011
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 7, 'Loss': 1.64313}: 100%|██████████| 164/164 [01:02<00:00,  2.63it/s]
Test Accuracy: 0.73162
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 8, 'Loss': 1.54832}: 100%|██████████| 164/164 [00:42<00:00,  3.83it/s]
Test Accuracy: 0.90883
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 9, 'Loss': 1.53205}: 100%|██████████| 164/164 [00:22<00:00,  7.37it/s]
Test Accuracy: 0.92051
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 10, 'Loss': 1.52779}: 100%|██████████| 164/164 [00:40<00:00,  4.01it/s]
Test Accuracy: 0.92549
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784

# uint8-custom 1
{'Epoch': 1, 'Loss': 2.30106}: 100%|██████████| 164/164 [00:54<00:00,  3.00it/s]
Test Accuracy: 0.10106
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 2, 'Loss': 2.13939}: 100%|██████████| 164/164 [00:44<00:00,  3.65it/s]
Test Accuracy: 0.38529
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 3, 'Loss': 1.98048}: 100%|██████████| 164/164 [00:23<00:00,  6.95it/s]
Test Accuracy: 0.44234
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 4, 'Loss': 1.82871}: 100%|██████████| 164/164 [00:43<00:00,  3.76it/s]
Test Accuracy: 0.64131
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 5, 'Loss': 1.77463}: 100%|██████████| 164/164 [00:25<00:00,  6.49it/s]
Test Accuracy: 0.68517
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 6, 'Loss': 1.75027}: 100%|██████████| 164/164 [00:24<00:00,  6.64it/s]
Test Accuracy: 0.70226
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 7, 'Loss': 1.73521}: 100%|██████████| 164/164 [00:47<00:00,  3.49it/s]
Test Accuracy: 0.71979
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 8, 'Loss': 1.71573}: 100%|██████████| 164/164 [00:44<00:00,  3.67it/s]
Test Accuracy: 0.73043
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 9, 'Loss': 1.68576}: 100%|██████████| 164/164 [00:53<00:00,  3.06it/s]
Test Accuracy: 0.73663
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
{'Epoch': 10, 'Loss': 1.64119}: 100%|██████████| 164/164 [00:21<00:00,  7.65it/s]
Test Accuracy: 0.8215
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  399.71174716949463